# Reducing ASD wavelengths error correlation matrix from `float64` to `float32`

Javier Gatón Herguedas

## Introduction

When interpolating the reflectance from the **CIMEL** points to the whole **ASD** spectrum, the ASD wavelengths error
correlation matrix is used to propagate the uncertainties. This is currently a **2151 × 2151** double-precision
floating-point (`float64`) matrix.

The `comet_maths` library requires **M** to be **positive semi-definite (PSD)**, which can be described
as **M** being a **Hermitian matrix** (including real symmetric matrices) where all its **eigenvalues are
real and non-negative**.

If the provided matrix is not **positive semi-definite**, `comet_maths` calculates the **closest positive
definite matrix** during run-time, which is a **time-consuming task** for a **2151 × 2151** matrix.

Using a **`float32` ASD error correlation matrix** instead of `float64` speeds up the simulation **2.5 times**. However, 
simply converting values to `float32` leads to truncation, causing the matrix to **lose its positive semi-definite property**. 
If we then compute the **nearest positive definite matrix** using `comet_maths.nearestPD_cholesky`, the resulting matrix has 
**diagonal values greater than 1**, which is **incompatible** with other uncertainty propagation functions.

### Objective
In this notebook, we explore different methods to obtain a **positive semi-definite `float32` matrix** 
with **1s in the diagonal**, derived from the original **`float64` positive semi-definite matrix**.

### Structure of this notebook

This notebook is structured in the following sections:
- **Introduction**: The problem, objective and structure of this notebook is explained.
- **Imports, data loads, and base function definitions**: Import the needed libraries, load the ASD matrix, define important base functions, constants, and variables.
- **Using comet_maths.nearestPD_cholesky**: Obtaining the nearest valid PSD matrix, using `comet_maths.nearestPD_cholesky`.
- **Using nearestPSD method**: Exploring a different method for obtaining the nearest valid PSD matrix.
  - **Method 0: closestPSD**: Explaining and using the basic `closestPSD`-based method, that yields a worse result than the method that depends on `nearestPD_cholesky`.
  - **Method 1: closestPSD_maxeigval**: A modification of the original method, where the maximum eighenvalue is taken into account when applying `closestPSD`'s inner tolerance check.
  - **Method 2: Applying closestPSD_maxeigval before and after truncating the matrix**: Applying `closestPSD_maxeigval` before and after truncating the matrix to 32 bits doesn't work very well, but with some modifications it ends up yielding a better result than the original `nearestPD_cholesky` method.
- **Store the new Matrix**: Use the best method explained in last section, and store the modified matrix in the correct file.


## Imports, data loads, and base function definitions

In [1]:
import warnings

import numpy as np
import xarray as xr
import comet_maths as cm

warnings.filterwarnings("ignore", message="Duplicate dimension names present")
warnings.filterwarnings("ignore", message="One of the provided covariance")

In [2]:
_DS_ASD_PATH = "./ds_ASD_original.nc"


def get_err_corr() -> np.ndarray:
    with xr.open_dataset(_DS_ASD_PATH) as ds:
        vals = ds["err_corr_reflectance_wavelength"].values
    return vals

In [3]:
def is_PSD(mat: np.ndarray, tol: float = 1e-10):
    """Check if a matrix is Positive Semi-Definite (PSD) by verifying eigenvalues."""
    eigenvalues = np.linalg.eigvalsh(mat)
    lim = -tol  # Allow for small numerical errors
    return np.all(eigenvalues >= lim)

In [4]:
# We need some tolerance due to floating point arithmetic.
# Even the original matrix wouldn't pass the is_PSD test without it
print(is_PSD(get_err_corr(), tol=0))
print(is_PSD(get_err_corr()))

False
True


In [5]:
def normalise_div(A: np.ndarray):
    return A / np.max(A)


def normalise(A: np.ndarray):
    A = A.copy()
    D = np.diag(1 / np.sqrt(np.diag(A)))
    A = D @ A @ D
    np.fill_diagonal(A, 1)
    return A

In [6]:
def analyze_ec(ec: np.ndarray, tol: float = 1e-10) -> dict:
    ec_src = get_err_corr()
    rel_diff = 100 * (ec - ec_src) / ec_src
    rel_diff = np.abs(np.ravel(rel_diff))
    analysis = {
        "dtype": ec.dtype,
        "PSD": is_PSD(ec, tol),
        "diff_min": min(rel_diff),
        "diff_max": max(rel_diff),
        "diff_mean": np.mean(rel_diff),
        "diff_std": np.std(rel_diff),
        "max_diag": max(np.diag(ec)),
    }
    return analysis


def describe_ec(ec: np.ndarray, tol: float = 1e-10):
    analysis = analyze_ec(ec, tol)
    vals = []
    for k in analysis:
        if k.startswith("diff"):
            vals.append(f"{k}: {analysis[k]:.4g}%")
        else:
            vals.append(f"{k}: {analysis[k]}")
    vals = "\n".join(vals)
    print(vals)

## Using comet_maths.nearestPD_cholesky

As described in the introduction, the original matrix is PSD

In [7]:
ec = get_err_corr()
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 0%
diff_max: 0%
diff_mean: 0%
diff_std: 0%
max_diag: 1.0000000000000002


But the `float32`-truncated matrix is not PSD

In [8]:
ec = get_err_corr().astype(np.float32)
describe_ec(ec)

dtype: float32
PSD: False
diff_min: 0%
diff_max: 5.932e-06%
diff_mean: 1.771e-06%
diff_std: 1.105e-06%
max_diag: 1.0


Using `comet_maths.nearestPD_cholesky` we get a PSD matrix, but the diagonal contains values too big (slightly over 1)

In [9]:
ec = get_err_corr().astype(np.float32)
ec = cm.nearestPD_cholesky(ec, return_cholesky=False)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 4.374e-12%
diff_max: 0.006771%
diff_mean: 1.166e-05%
diff_std: 0.0001453%
max_diag: 1.0000677108764648


If we normalise that matrix, we get a valid PSD matrix.

In [10]:
ec = get_err_corr().astype(np.float32)
ec = cm.nearestPD_cholesky(ec, return_cholesky=False)
ec = normalise(ec)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.006812%
diff_mean: 0.006736%
diff_std: 0.0001459%
max_diag: 1.0


The mean relative difference is 0.006736%. Can we get a more similar PSD matrix?

## Using a nearestPSD method

The most popular method is a similar version of the comet_maths method, but there are multiple
methods online for the calculation of the nearest PSD. Most of them haven't been useful for the
purpose of this notebook, except for a modification of the method described in the Method 0 subsection.

### Method 0: closestPSD

In [11]:
def closestPSD_0(A: np.ndarray, tol=1e-10) -> np.ndarray:
    A = (A + A.T) / 2
    eigvals, eigvecs = np.linalg.eigh(A)
    eigvals[eigvals < tol] = 0
    A = eigvecs @ np.diag(eigvals) @ eigvecs.T
    return A

In [12]:
ec = get_err_corr().astype(np.float32)
ec = closestPSD_0(ec)
describe_ec(ec)

dtype: float32
PSD: False
diff_min: 4.419e-12%
diff_max: 4.768e-05%
diff_mean: 4.721e-06%
diff_std: 3.674e-06%
max_diag: 1.0000004768371582


It doesn't work. In Method 1 we explore a different approach for this method.

### Method 1: closestPSD_maxeigval

If we look at Brian Borchers' answer in stackexchange: https://scicomp.stackexchange.com/a/12984,
we see that a good definition for the `is_PSD` method is to, instead of checking that
all eigenvalues are greater than `-tol`, check that they are greater than `-tol * max(eigenvals)`.

We aren't modifying our `is_PSD` method, but if we apply that idea to this function it starts working.
This can be seen in the Method 1 subsection.

In [13]:
def closestPSD_maxeigval(A: np.ndarray, tol=1e-10) -> np.ndarray:
    A = (A + A.T) / 2
    eigvals, eigvecs = np.linalg.eigh(A)
    eigvals[eigvals < tol] = tol * max(eigvals)
    A = eigvecs @ np.diag(eigvals) @ eigvecs.T
    return A

We increase the tolerance until the resulting matrix is actually PSD, but the diagonal values are too high.

In [14]:
ec = get_err_corr().astype(np.float32)
for i in range(10):
    eci = closestPSD_maxeigval(ec, tol=10 ** (-10 + i))
    if is_PSD(eci):
        break
ec = eci
print(f"1e-{10-i}")
describe_ec(ec)

1e-6
dtype: float32
PSD: True
diff_min: 6.162e-12%
diff_max: 0.1769%
diff_mean: 0.001306%
diff_std: 0.003554%
max_diag: 1.0017693042755127


We normalise the matrix to get a valid diagonal, and we get a valid PSD matrix but the mean relative difference is much worse than with the `nearestPD_cholesky`
method.

It was 0.006736%, now it's 0.1575%.

In [15]:
ec = get_err_corr().astype(np.float32)
ec = closestPSD_maxeigval(ec, tol=1e-6)
ec = normalise(ec)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.1827%
diff_mean: 0.1575%
diff_std: 0.01085%
max_diag: 1.0


### Method 2: Applying closestPSD_maxeigval before and after truncating the matrix

If the function is applied to the original matrix before applying it to the truncated one, we
obtain slightly better results, but depending on the original ASD matrix, the final normalised
matrix might not be a valid PSD matrix.

In [16]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec)
describe_ec(ec)
ec = ec.astype(np.float32)
describe_ec(ec)
for i in range(10):
    for j in range(9):
        eci = closestPSD_maxeigval(ec, tol=(j + 1) * 10 ** (-10 + i))
        if is_PSD(eci):
            break
    if is_PSD(eci):
        break
ec = eci
print(f"\n1e-{10-i}")
describe_ec(ec)
ec = normalise(ec)
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 0%
diff_max: 1.848e-05%
diff_mean: 4.098e-08%
diff_std: 3.907e-07%
max_diag: 1.0000001848256679
dtype: float32
PSD: False
diff_min: 4.374e-12%
diff_max: 2.384e-05%
diff_mean: 1.781e-06%
diff_std: 1.186e-06%
max_diag: 1.000000238418579

1e-7
dtype: float32
PSD: True
diff_min: 6.339e-12%
diff_max: 0.1539%
diff_mean: 0.001476%
diff_std: 0.002907%
max_diag: 1.001538634300232
dtype: float32
PSD: False
diff_min: 0%
diff_max: 0.1603%
diff_mean: 0.1251%
diff_std: 0.009112%
max_diag: 1.0


If we apply the function to the original matrix but with an increased tolerance, the results improve.
Usually a tolerance of 1e-8 yields better results than the original `nearestPD_cholesky` method.

We obtain a mean relative difference of 0.001762%, which is more than 5 times better than the 0.006736% mrd.

In [17]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec, 1e-8)
describe_ec(ec)
ec = ec.astype(np.float32)
describe_ec(ec)
for i in range(10):
    for j in range(9):
        eci = closestPSD_maxeigval(ec, tol=(j + 1) * 10 ** (-10 + i))
        if is_PSD(eci):
            break
    if is_PSD(eci):
        break
ec = eci
print(f"\n1e-{10-i}")
describe_ec(ec)
ec = normalise(ec)
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 8.574e-14%
diff_max: 0.001848%
diff_mean: 4.098e-06%
diff_std: 3.907e-05%
max_diag: 1.0000184825667844
dtype: float32
PSD: True
diff_min: 5.356e-12%
diff_max: 0.001848%
diff_mean: 5.048e-06%
diff_std: 3.9e-05%
max_diag: 1.0000184774398804

1e-10
dtype: float32
PSD: True
diff_min: 5.385e-12%
diff_max: 0.001872%
diff_mean: 7.62e-06%
diff_std: 3.907e-05%
max_diag: 1.0000187158584595
dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.001875%
diff_mean: 0.001762%
diff_std: 0.0001148%
max_diag: 1.0


Through trial and error the value 4e-9 was selected, instead of 1e-8 (this varies depending on the original ASD matrix).

A valid PSD matrix with a mean relative difference of 0.0006985% was obtained. This is around 10 times
smaller than the matrix adapted initally.

In [18]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec, 4e-9)  # 4e-9, 0.0006985 (0.0007764)
describe_ec(ec)
ec = ec.astype(np.float32)
describe_ec(ec)
if not is_PSD(ec):
    for i in range(10):
        for j in range(9):
            eci = closestPSD_maxeigval(ec, tol=(j + 1) * 10 ** (-10 + i))
            if is_PSD(eci):
                break
        if is_PSD(eci):
            break
    ec = eci
    print(f"{(j+1)}e-{10-i}")
    describe_ec(ec)
ec = normalise(ec)
describe_ec(ec)
if not is_PSD(ec):
    for i in range(10):
        for j in range(9):
            eci = closestPSD_maxeigval(ec, tol=(j + 1) * 10 ** (-10 + i))
            eci = normalise(eci)
            if is_PSD(eci):
                break
        if is_PSD(eci):
            break
    ec = eci
    print(f"{j+1}e-{10-i}")
    describe_ec(ec)
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 0%
diff_max: 0.0007393%
diff_mean: 1.639e-06%
diff_std: 1.563e-05%
max_diag: 1.000007393026714
dtype: float32
PSD: True
diff_min: 5.356e-12%
diff_max: 0.0007391%
diff_mean: 2.852e-06%
diff_std: 1.562e-05%
max_diag: 1.0000073909759521
dtype: float32
PSD: False
diff_min: 0%
diff_max: 0.0007566%
diff_mean: 0.0007055%
diff_std: 4.748e-05%
max_diag: 1.0
2e-10
dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.0007764%
diff_mean: 0.0006985%
diff_std: 4.756e-05%
max_diag: 1.0
dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.0007764%
diff_mean: 0.0006985%
diff_std: 4.756e-05%
max_diag: 1.0


## Store the new matrix

In [19]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec, tol=4e-9)
ec = ec.astype(np.float32)
ec = normalise(ec)
ec = closestPSD_maxeigval(ec, tol=2e-10)
ec = normalise(ec)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.0007764%
diff_mean: 0.0006985%
diff_std: 4.756e-05%
max_diag: 1.0


In [20]:
ds = xr.open_dataset(_DS_ASD_PATH)
ds["err_corr_reflectance_wavelength"].values = ec
ds.to_netcdf("./ds_ASD_32.nc")
ds.close()